In [1]:
pip install darts

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import fosforml
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [3]:
my_session.connection.database

'ASSORTMENT_PLANNING'

In [4]:
my_session.connection.schema

'CPG_BRONZE'

In [5]:
table_name = "ASSORTMENT_PLANNING.CPG_BRONZE.SALES_CLEAN"

In [6]:
sf_df = my_session.sql("select * from {}".format(table_name))
type(sf_df)
df=sf_df.to_pandas()
type(df)

pandas.core.frame.DataFrame

In [7]:
df.head(5)

,MNTH_CODE,TRANS_DATE,START_DATE,SALES_VALUE,SALES_UNITS,SALES_VOLUME,SALES_PTR_VALUE,OC_CODE,DISTRIBUTOR_CODE,OUTLET_CODE,CITY,STATE,COUNTY,STREET,PRODUCT_CODE,CATEGORY,SUBCATEGORY,BRAND
0,202312,2023-12-26,2023-11-27,372.73,2,0.000650,372.727273,202212,DB0110,OL112848,Carbondale,Illinois,City Center,Str2,PRD0118,Soap,Medicated Soap,Burgundy
1,202312,2023-12-26,2023-11-27,436.36,3,0.000900,436.363636,202212,DB0110,OL112848,Carbondale,Illinois,City Center,Str2,PRD0131,Soap,Toilet Soap,Grape vine
2,202312,2023-12-24,2023-11-27,202.82,2,0.000214,209.090909,202212,DB0209,OL112555,Shelbyville,Tennessee,City Center,Str5,PRD0065,Hair Care,Hair Oil,Crimson
3,202312,2023-12-29,2023-11-27,142.86,16,0.000384,142.857143,202212,DB0655,OL113173,Renton,Washington,Orange,Str2,PRD0106,Hair Care,Shampoo,Hair & Shoulders
4,202312,2023-12-17,2023-11-27,145.45,1,0.000300,145.454545,202212,DB0652,OL112373,Wallingford,Connecticut,Dolphin,Str1,PRD0070,Perfume and Deodrants,Female Deodrant,Pista green


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966400 entries, 0 to 966399
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   MNTH_CODE         966400 non-null  int32  
 1   TRANS_DATE        966400 non-null  object 
 2   START_DATE        966400 non-null  object 
 3   SALES_VALUE       966400 non-null  float64
 4   SALES_UNITS       966400 non-null  int16  
 5   SALES_VOLUME      966400 non-null  float64
 6   SALES_PTR_VALUE   966400 non-null  float64
 7   OC_CODE           966400 non-null  int32  
 8   DISTRIBUTOR_CODE  966400 non-null  object 
 9   OUTLET_CODE       966400 non-null  object 
 10  CITY              966400 non-null  object 
 11  STATE             966400 non-null  object 
 12  COUNTY            966400 non-null  object 
 13  STREET            966400 non-null  object 
 14  PRODUCT_CODE      966400 non-null  object 
 15  CATEGORY          966400 non-null  object 
 16  SUBCATEGORY       96

## Generate Additional Features

In [9]:
import pandas as pd

def generate_features(df):
    # Ensure the TRANS_DATE column is in datetime format
    df['TRANS_DATE'] = pd.to_datetime(df['TRANS_DATE'])
    
    # Extract temporal features
    df['year'] = df['TRANS_DATE'].dt.year
    df['month'] = df['TRANS_DATE'].dt.month
    df['day'] = df['TRANS_DATE'].dt.day
    df['dayofweek'] = df['TRANS_DATE'].dt.dayofweek
    df['quarter'] = df['TRANS_DATE'].dt.quarter
    df['is_month_start'] = df['TRANS_DATE'].dt.is_month_start
    df['is_month_end'] = df['TRANS_DATE'].dt.is_month_end
    
    # Create UNIT_PTR feature
    df['UNIT_PTR'] = df['SALES_PTR_VALUE'] / df['SALES_UNITS']
    
    return df

In [10]:
df = generate_features(df)

In [11]:
df.head()

,MNTH_CODE,TRANS_DATE,START_DATE,SALES_VALUE,SALES_UNITS,SALES_VOLUME,SALES_PTR_VALUE,OC_CODE,DISTRIBUTOR_CODE,OUTLET_CODE,...,SUBCATEGORY,BRAND,year,month,day,dayofweek,quarter,is_month_start,is_month_end,UNIT_PTR
0,202312,2023-12-26,2023-11-27,372.73,2,0.000650,372.727273,202212,DB0110,OL112848,...,Medicated Soap,Burgundy,2023,12,26,1,4,False,False,186.363636
1,202312,2023-12-26,2023-11-27,436.36,3,0.000900,436.363636,202212,DB0110,OL112848,...,Toilet Soap,Grape vine,2023,12,26,1,4,False,False,145.454545
2,202312,2023-12-24,2023-11-27,202.82,2,0.000214,209.090909,202212,DB0209,OL112555,...,Hair Oil,Crimson,2023,12,24,6,4,False,False,104.545455
3,202312,2023-12-29,2023-11-27,142.86,16,0.000384,142.857143,202212,DB0655,OL113173,...,Shampoo,Hair & Shoulders,2023,12,29,4,4,False,False,8.928571
4,202312,2023-12-17,2023-11-27,145.45,1,0.000300,145.454545,202212,DB0652,OL112373,...,Female Deodrant,Pista green,2023,12,17,6,4,False,False,145.454545


In [12]:
df = df.sort_values(by='TRANS_DATE')

In [13]:
# Convert all column names to uppercase
df.columns = [col.upper() for col in df.columns]

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 966400 entries, 787515 to 905006
Data columns (total 26 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   MNTH_CODE         966400 non-null  int32         
 1   TRANS_DATE        966400 non-null  datetime64[ns]
 2   START_DATE        966400 non-null  object        
 3   SALES_VALUE       966400 non-null  float64       
 4   SALES_UNITS       966400 non-null  int16         
 5   SALES_VOLUME      966400 non-null  float64       
 6   SALES_PTR_VALUE   966400 non-null  float64       
 7   OC_CODE           966400 non-null  int32         
 8   DISTRIBUTOR_CODE  966400 non-null  object        
 9   OUTLET_CODE       966400 non-null  object        
 10  CITY              966400 non-null  object        
 11  STATE             966400 non-null  object        
 12  COUNTY            966400 non-null  object        
 13  STREET            966400 non-null  object        
 14  PROD

In [17]:
df_sorted = df.sort_values(by=['OUTLET_CODE', 'PRODUCT_CODE', 'TRANS_DATE'])

In [19]:
pd.set_option('display.max_columns', None)

In [24]:
# List of columns in the desired order
columns_order = ['OUTLET_CODE', 'PRODUCT_CODE', 'TRANS_DATE', 'UNIT_PTR'] + [col for col in df_sorted.columns if col not in ['OUTLET_CODE', 'PRODUCT_CODE']]

# Reorder the DataFrame columns
df_sorted = df_sorted[columns_order]

In [25]:
df_sorted.tail()

,OUTLET_CODE,PRODUCT_CODE,TRANS_DATE,UNIT_PTR,MNTH_CODE,TRANS_DATE,START_DATE,SALES_VALUE,SALES_UNITS,SALES_VOLUME,SALES_PTR_VALUE,OC_CODE,DISTRIBUTOR_CODE,CITY,STATE,COUNTY,STREET,CATEGORY,SUBCATEGORY,BRAND,YEAR,MONTH,DAY,DAYOFWEEK,QUARTER,IS_MONTH_START,IS_MONTH_END,UNIT_PTR
657602,OL97624,PRD0105,2024-01-12,4.464286,202401,2024-01-12,2024-01-02,53.57,12,0.000144,53.571429,202201,DB0706,Virden,Illinois,Sumter,Str2,Perfume and Deodrants,Female Perfume,Garnet,2024,1,12,4,1,False,False,4.464286
381675,OL97624,PRD0105,2024-02-09,4.464286,202402,2024-02-09,2024-01-30,53.57,12,0.000144,53.571429,202202,DB0706,Virden,Illinois,Sumter,Str2,Perfume and Deodrants,Female Perfume,Garnet,2024,2,9,4,1,False,False,4.464286
311853,OL97624,PRD0105,2024-03-23,4.464286,202403,2024-03-23,2024-02-27,53.57,12,0.000144,53.571429,202203,DB0706,Virden,Illinois,Sumter,Str2,Perfume and Deodrants,Female Perfume,Garnet,2024,3,23,5,1,False,False,4.464286
758426,OL97624,PRD0105,2024-04-20,4.464286,202404,2024-04-20,2024-04-03,53.57,12,0.000144,53.571429,202204,DB0706,Virden,Illinois,Sumter,Str2,Perfume and Deodrants,Female Perfume,Garnet,2024,4,20,5,2,False,False,4.464286
278374,OL97624,PRD0106,2024-03-16,8.928571,202403,2024-03-16,2024-02-27,142.86,16,0.000400,142.857143,202203,DB0706,Virden,Illinois,Sumter,Str2,Hair Care,Shampoo,Hair & Shoulders,2024,3,16,5,1,False,False,8.928571


Split the dataset into test and train

In [26]:
# Split the data
train_data = df[df['MNTH_CODE'] != 202408]
test_data = df[df['MNTH_CODE'] == 202408]

In [16]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891440 entries, 787515 to 448089
Data columns (total 26 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   MNTH_CODE         891440 non-null  int32         
 1   TRANS_DATE        891440 non-null  datetime64[ns]
 2   START_DATE        891440 non-null  object        
 3   SALES_VALUE       891440 non-null  float64       
 4   SALES_UNITS       891440 non-null  int16         
 5   SALES_VOLUME      891440 non-null  float64       
 6   SALES_PTR_VALUE   891440 non-null  float64       
 7   OC_CODE           891440 non-null  int32         
 8   DISTRIBUTOR_CODE  891440 non-null  object        
 9   OUTLET_CODE       891440 non-null  object        
 10  CITY              891440 non-null  object        
 11  STATE             891440 non-null  object        
 12  COUNTY            891440 non-null  object        
 13  STREET            891440 non-null  object        
 14  PROD